In [166]:
import numpy as np
import scipy.io as sio
import open3d as o3d

# Load the .mat file
datapath = "xyz5.mat"
mat = sio.loadmat(datapath)

# grid_size = (24, 24, 24)
# grid_size = (32, 32, 32)
# grid_size = (48, 48, 48)
grid_size = (64, 64, 64)

In [167]:
npxyz = np.asarray(mat['xyz'])
print(type(npxyz))
print(npxyz.shape)
print(npxyz)

<class 'numpy.ndarray'>
(41935, 3)
[[   2   -1    1]
 [   9   -1    1]
 [  10   -1    1]
 ...
 [ 314 -139  357]
 [ 317 -139  357]
 [ 318 -139  357]]


In [168]:
npcolor = np.asarray(mat['color'])
print(type(npcolor))
print(npcolor.shape)
print(npcolor)


<class 'numpy.ndarray'>
(41935, 3)
[[255 255   0]
 [255 255   0]
 [255 255   0]
 ...
 [255   0   0]
 [255   0   0]
 [255   0   0]]


In [169]:
# Assuming the point cloud data is stored in a variable named 'pointCloud'
points = mat['xyz']
color = mat['color']

# Create an Open3D point cloud object
point_cloud = o3d.geometry.PointCloud()

# Assign colors to the point cloud object
# Open3D expects color values to be in the range [0, 1]
color = color / 255.0
point_cloud.colors = o3d.utility.Vector3dVector(color)

# Assign points to the point cloud object
point_cloud.points = o3d.utility.Vector3dVector(points)

# fit to unit cube. 
point_cloud.scale( (grid_size[0] - 1) / (np.max(point_cloud.get_max_bound() - point_cloud.get_min_bound())),
          center=point_cloud.get_center())

# Visualize the point cloud
# o3d.visualization.draw_geometries([point_cloud])
print(point_cloud.get_max_bound())
print(point_cloud.get_min_bound())

[258.15198569 -19.41922785 130.82539454]
[195.15198569 -35.97922785  88.10539454]


In [170]:
# Create a voxel grid from the point cloud with a voxel_size of 0.01
voxel_grid=o3d.geometry.VoxelGrid.create_from_point_cloud(point_cloud,voxel_size=1)

# print(voxel_grid.check_if_included(point_cloud.points))
# voxel_grid.get_center()
# voxel_grid.dimension()
# voxel_grid.get_geometry_type()
# voxel_grid.get_voxel([0,0,0])
# np.asarray(voxel_grid.get_voxel_bounding_points([0,0,0]))
# voxel_grid.get_voxel_center_coordinate([0,0,0])
# voxel_grid.has_voxels()
# voxel_grid.is_empty()
# voxel_grid.voxel_size
len(voxel_grid.get_voxels())

1154

In [171]:

min_bound = voxel_grid.get_min_bound()
max_bound = voxel_grid.get_max_bound()
voxel_size = (max_bound - min_bound) / np.array(grid_size)
print(voxel_size)
print(min_bound, max_bound)

[1.      0.28125 0.6875 ]
[194.65198569 -36.47922785  87.60539454] [258.65198569 -18.47922785 131.60539454]


In [177]:

occupancy_grid = np.zeros(grid_size, dtype=bool)
for voxel in voxel_grid.get_voxels():
    print(voxel.grid_index)
    index = voxel.grid_index.astype(int)
    if np.all(index >= 0) and np.all(index < grid_size):
        occupancy_grid[tuple(index)] = True
        print(index)


[10  5  1]
[10  5  1]
[36 14 40]
[36 14 40]
[ 0 17  0]
[ 0 17  0]
[53  6  1]
[53  6  1]
[52 17  0]
[52 17  0]
[35 17  6]
[35 17  6]
[42 17  0]
[42 17  0]
[31 14 38]
[31 14 38]
[15 17  2]
[15 17  2]
[ 8 17  0]
[ 8 17  0]
[ 3 17  0]
[ 3 17  0]
[49 16  0]
[49 16  0]
[63 13  2]
[63 13  2]
[ 6 17  0]
[ 6 17  0]
[ 9 17  0]
[ 9 17  0]
[ 1 17  0]
[ 1 17  0]
[13 17  6]
[13 17  6]
[18 17  1]
[18 17  1]
[ 2 17  0]
[ 2 17  0]
[31 17  5]
[31 17  5]
[63 17  2]
[63 17  2]
[ 0 13  0]
[ 0 13  0]
[ 4 17  0]
[ 4 17  0]
[18 17  2]
[18 17  2]
[16 17  0]
[16 17  0]
[30 17 12]
[30 17 12]
[40 10  1]
[40 10  1]
[59 17  0]
[59 17  0]
[24 17  0]
[24 17  0]
[63  9  4]
[63  9  4]
[12  6  0]
[12  6  0]
[ 5 17  0]
[ 5 17  0]
[39 16  0]
[39 16  0]
[16 14 38]
[16 14 38]
[26 17  0]
[26 17  0]
[19 10  1]
[19 10  1]
[ 7 17  0]
[ 7 17  0]
[27 17  1]
[27 17  1]
[10 17  0]
[10 17  0]
[ 9 17  2]
[ 9 17  2]
[52 16  0]
[52 16  0]
[26 17  9]
[26 17  9]
[12 17  0]
[12 17  0]
[15 14 23]
[15 14 23]
[15 17  0]
[15 17  0]
[21 17  0]

In [173]:
voxel_centers = []
grid_size = occupancy_grid.shape
for x in range(grid_size[0]):
    for y in range(grid_size[1]):
        for z in range(grid_size[2]):
            if occupancy_grid[x, y, z]:
                voxel_centers.append([x, y, z])
voxel_centers = np.array(voxel_centers)

print((voxel_centers.shape))
print((occupancy_grid.shape))

(1154, 3)
(64, 64, 64)


In [174]:
# # Create a VoxelGrid from the voxel centers
# if len(voxel_centers) > 0:
#     voxel_centers = o3d.utility.Vector3dVector(voxel_centers)
#     new_voxel_grid = o3d.geometry.VoxelGrid.create_from_points(voxel_centers, voxel_size)
print(type(voxel_size))

<class 'numpy.ndarray'>


In [175]:
if len(voxel_centers) > 0:
    point_cloud2 = o3d.geometry.PointCloud()
    point_cloud2.points = o3d.utility.Vector3dVector(voxel_centers)
    
    # Create a VoxelGrid from the PointCloud
    new_voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(point_cloud2, voxel_size=0.7)

    print(len(new_voxel_grid.get_voxels()))


1154


In [176]:
# Initialize a visualizer object
vis = o3d.visualization.Visualizer()
# Create a window, name it and scale it
vis.create_window(window_name='dummy Visualize', width=1400, height=900)

# Add the voxel grid to the visualizer
vis.add_geometry(new_voxel_grid)

# We run the visualizater
vis.run()
# Once the visualizer is closed destroy the window and clean up
vis.destroy_window()